# Setup

In [1]:
import pandas as pd
import pytz

SAVE_CSV_SEPARATOR = ';'

def save(data, path_base, csv=True, xls=True):
    if data.index.tz is None:
        raise ValueError('Timezone must be explicit!')
    data.index.name = 'Time (UTC)'
    data = data.tz_convert(tz=pytz.UTC).tz_localize(tz=None)
    if csv:
        data.to_csv(path_base + '.csv', sep=SAVE_CSV_SEPARATOR)
    if xls:
        data.to_excel(path_base + '.xls')
    
    

# Göteborg Energi

In [2]:
data = pd.read_csv(
    r'\\sp.se\Dok\ET\FoI\EU\FP7\Step-Up\WP2\Energimodell\GE via CTH\GE 2010-01-01 - 2014-03-09 (lokal tid).csv',
    index_col=0,
    sep=';',
    encoding='utf-8',
    parse_dates=True)

# Data is (implicitly) given in wall-clock time, i.e. normal time and daylight saving time mixed.
# Sort it out:
data = data.tz_localize(tz='Europe/Stockholm', axis='rows', ambiguous='infer')

# Shift one hour backwards. Original data states production at end of period.
# I want production at start of period, e.g. data at 01:00 is the production 01:00-02:00 in the output of this script.
data.shift(-1, freq='H') 

save(data, 'cleaned/GE 2010-01-01 - 2014-03-09 (UTC)', xls=True, csv=True)

# Svenska Kraftnät

In [3]:
data = pd.read_csv(
    r'\\sp.se\Dok\ET\FoI\EU\FP7\Step-Up\WP2\Energimodell\Elleverans SKN\SKN 2001-01-01 - 2014-01-31.csv',
    index_col=0,
    sep=';',
    encoding='utf-8',
    parse_dates=True)

data = data.tz_localize(tz=pytz.UTC)

save(data, 'cleaned/SKN 2001-01-01 - 2014-01-31 (UTC)', xls=False, csv=True)

# Elspot

In [5]:
from pandas.tseries.offsets import DateOffset
import re
import math

# Elspot data has a horrible format.
# It is given in local "wall-clock" time, i.e. implicit normal/daylight saving time.
# Hour 02.00-03.00 is blank in the data file the last Sunday in March.
# Hour 02.00-03.00 should be read TWICE the last Sunday in October.
# The (price) value should be the same two hours in a row.

def read_raw_data(path, year):
    data = pd.read_excel(path, skiprows=list(range(5)), index_col=0)
    data = data[data.index.year == year]
    data.dropna(thresh=24, inplace=True)
    assert 365 <= len(data) <= 366
    assert all(int(col) == j+1 for j, col in enumerate(data.columns[0:24]))
    return data

def interpret(raw_data):
    d = []
    dst_started = dst_ended = False
    for i, row in raw_data.iterrows():
        if re.match('[Ss][uo]mm[ae]rti(d|me)', str(row.iloc[28])):
            assert not dst_started
            dst_started = True
            assert math.isnan(row.iloc[2])
            hours = list(range(2)) + list(range(3,24)) # Skip non-existing hour
        elif re.match('[WwVv]interti(d|me)', str(row[28])):
            assert not dst_ended
            dst_ended = True
            hours = list(range(3)) + list(range(2,24)) # Duplicate double hour
        else:
            hours = list(range(24))

        items = row.iloc[hours]        
        for hour, value in zip(hours, items):
            timestamp = row.name + DateOffset(hours=hour)
            d.append((timestamp, value))

    index, data = zip(*d)

    series = pd.Series(data=data, index=index)

    # Data is now implicitly given in wall-clock time, i.e. normal time and daylight saving time mixed.
    # Sort it out:
    series = series.tz_localize(tz='Europe/Stockholm', axis='rows', ambiguous='infer')
    
    return pd.DataFrame.from_dict({'Price': series})


all_elspot = []
years = tuple(range(2000, 2014))
for year in years:
    d = read_raw_data(
        r'\\sp.se\Dok\ET\Gemensamt\Övrigt\Data Nord Pool FTP\Elspot\Elspot_prices\Sweden\{}\stosek{}.xls'.format(year, str(year)[2:]), year)
    d = interpret(d)
    all_elspot.append(d)
    
save(
    pd.concat(all_elspot),
    'cleaned/Elspot Stockholm {} - {} (UTC)'.format(years[0], years[-1]),
    xls=False)

